In [ ]:
# stdlib
from collections import Counter
import os

# syft absolute
import syft as sy
from syft.util.test_helpers.email_helpers import get_email_server
from syft.util.test_helpers.email_helpers import load_users

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
ROOT_EMAIL, ROOT_PASSWORD = "admin@bigquery.org", "bqpw"
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
low_port = os.environ.get("CLUSTER_HTTP_PORT_LOW", "9083")
print(environment, low_port)

# Launch server & login

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=4,
    create_producer=True,
    port=low_port,
)

In [ ]:
email_server, smtp_server = get_email_server()

In [ ]:
low_client = sy.login(
    url=f"http://localhost:{low_port}", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
users = load_users(low_client)

# Create jobs

In [ ]:
# syft absolute
from syft.util.test_helpers.job_helpers import TestJob
from syft.util.test_helpers.job_helpers import create_jobs
from syft.util.test_helpers.job_helpers import extract_code_path

# Inspect job data (requests for these jobs to be created)

In [ ]:
num_jobs = int(os.environ.get("NUM_TEST_JOBS", 10))

jobs_data = create_jobs(users, total_jobs=num_jobs)

In [ ]:
counts = Counter([j.job_type for j in jobs_data])
for k, v in counts.most_common():
    print(f"{k}: #{v}")

In [ ]:
# syft absolute
from syft.util.test_helpers.job_helpers import save_jobs

In [ ]:
print(f"{len(jobs_data)=}")

for job in jobs_data:
    print(f"{job.job_type=}, {job.should_succeed=}, {job.should_submit=}")

In [ ]:
assert len(jobs_data) == num_jobs
assert all(isinstance(j, TestJob) for j in jobs_data)
assert all(job.client is not None for job in jobs_data)

In [ ]:
save_jobs(jobs_data)

# Submit jobs


In [ ]:
admin_emails_before = len(email_server.get_emails_for_user("admin@bigquery.org"))
print(f"{admin_emails_before=}")

## Test Succesful jobs

In [ ]:
jobs_submit_should_succeed = [j for j in jobs_data if j.should_submit]

In [ ]:
for job in jobs_submit_should_succeed:
    client = job.client
    response = client.api.services.bigquery.submit_query(
        func_name=job.func_name, query=job.query
    )
    job.code_path = extract_code_path(response)

## Test failures

In [ ]:
jobs_submit_should_fail = [j for j in jobs_data if not j.should_submit]

In [ ]:
for job in jobs_submit_should_fail:
    client = job.client

    with sy.raises(sy.SyftException):
        client.api.services.bigquery.submit_query(
            func_name=job.func_name, query=job.query
        )

In [ ]:
for job in jobs_data:
    print(f"Job {job.func_name:.20} {job.should_submit=}, {job.is_submitted=}")

assert all(job.is_submitted == job.should_submit for job in jobs_data)

In [ ]:
save_jobs(jobs_data)

## Test: cannot execute submitted jobs yet

In [ ]:
submitted_jobs = [job for job in jobs_data if job.should_submit]
job_execution_fns = [getattr(job.client.code, job.code_path) for job in submitted_jobs]
assert len(submitted_jobs)  # failsafe for next tests

In [ ]:
for fn in job_execution_fns:
    # blocking
    with sy.raises(
        sy.SyftException(public_message="*Your code is waiting for approval*")
    ):
        result = fn()

In [ ]:
# currently errors out with
# syft.types.errors.SyftException: Please wait for the admin to allow the execution of this code

for fn in job_execution_fns:
    # nonblocking
    with sy.raises(
        sy.SyftException(
            public_message="*Please wait for the admin to allow the execution of this code*"
        )
    ):
        result_job = fn(blocking=False)
        result_job.wait()

# Verify that admin has emails for submitted requests

In [ ]:
num_should_submit = sum(j.should_submit for j in jobs_data)
admin_emails_after = len(email_server.get_emails_for_user(ROOT_EMAIL))
print("admin emails after", admin_emails_after)
assert admin_emails_after >= admin_emails_before + num_should_submit

# Submit a broken query

In [ ]:
ds_client = users[0].client
submission = ds_client.api.services.bigquery.submit_query(
    func_name="broken_query", query="BROKEN QUERY"
)
submission

# Cleanup

In [ ]:
if environment != "remote":
    server_low.land()
smtp_server.stop()